In [ ]:
import requests
from bs4 import BeautifulSoup as BS

In [ ]:
import requests
import mysql.connector
from datetime import datetime
from dateutil.parser import parse

In [ ]:
import stuff
cnx = mysql.connector.connect(
    host = stuff.host,
    user = stuff.user,
    passwd = stuff.password)

In [ ]:
cursor = cnx.cursor()

In [ ]:
#connect to database in mysql. I called the database for this project 'movie_project'
cnx.database = 'movie_project'

In [213]:
#url request. Must insert url with quotations "url"
def request_func (url):
    page = requests.get(url)
    soup = BS(page.content, 'html.parser')
    return soup

In [214]:
#getting movie titles
def movie_titles (soup):
    names=soup.find_all('td', class_='a-text-left mojo-field-type-release mojo-cell-wide')
    All_names = []
    for thing in names:
        try:
            All_names.append(thing.string)
        except:
            All_names.append('NAN')
            continue
    return All_names

In [215]:
#extracting total gross $ for each movie
def total_gross (soup):
    gross_opening=soup.find_all('td', class_='a-text-right mojo-field-type-money')
    i=0
    gross=[]
    opening=[]
    for thing in gross_opening:
        if i%2==0:
            gross.append(gross_opening[i].string)
        elif i%2!=0:
            opening.append(gross_opening[i].string)
        i+=1
    gross_numbers=[]
    for item in gross:
        maybe = item.strip('$')
        maybe2=maybe.strip(',')
        maybe3 = maybe2.replace(',','')
        maybe4=int(maybe3)
        gross_numbers.append(maybe4)
    #changes dashes to Nans
    gross_numbers_with_none=[]
    for item in gross_numbers:
        if type(item)==int:
            gross_numbers_with_none.append(item)
        else:
            gross_numbers_with_none.append('NAN')
            continue
    return gross_numbers_with_none

In [216]:
#opening weekend gross $
def opening_weekend_gross (soup):
    gross_opening=soup.find_all('td', class_='a-text-right mojo-field-type-money')
    i=0
    gross=[]
    opening=[]
    for thing in gross_opening:
        if i%2==0:
            gross.append(gross_opening[i].string)
        elif i%2!=0:
            opening.append(gross_opening[i].string)
        i+=1
    opening_numbers=[]
    for item in opening:
        try:
            maybe = item.strip('$')
            maybe2=maybe.strip(',')
            maybe3 = maybe2.replace(',','')
            maybe4=int(maybe3)
            opening_numbers.append(maybe4)
        except:
            opening_numbers.append(item)
            
    opening_numbers_with_none=[]
    for item in opening_numbers:
        if type(item)==int:
            opening_numbers_with_none.append(item)
        else:
            opening_numbers_with_none.append('NAN')
            continue
    return opening_numbers_with_none

In [217]:
#obtaining release dates
def release_dates (soup):
    opening_date=soup.find_all('td', class_='a-text-left mojo-field-type-date mojo-sort-column a-nowrap')
    print(opening_date[0])
    All_opening_dates = []
    for thing in opening_date:
        try:
            All_opening_dates.append(thing.string)
        except:
            All_opening_dates.append('NAN')
            continue
            
    release_date_clean=[]
    for thing in All_opening_dates:
        y=thing.replace('Jan ','2018-01-')
        y2=y.replace('Feb ','2018-02-')
        y3=y2.replace('Mar ','2018-03-')
        y4=y3.replace('Apr ','2018-04-')
        y5=y4.replace('May ','2018-05-')
        y6=y5.replace('Jun ','2018-06-')
        y7=y6.replace('Jul ','2018-07-')
        y8=y7.replace('Aug ','2018-08-')
        y9=y8.replace('Sep ','2018-09-')
        y10=y9.replace('Oct ','2018-10-')
        y11=y10.replace('Nov ','2018-11-')
        y12=y11.replace('Dec ','2018-12-')
        release_date_clean.append(y12)
        
    release_date_date_time=[]
    for thing in release_date_clean:
            y2=datetime.strptime(thing,'%Y-%m-%d')
            release_date_date_time.append(y2)
            
    return release_date_date_time

In [218]:
#obtaining closing dates of movies in movie theater
def closing_dates (soup):
    closing_date=soup.find_all('td', class_='a-text-left mojo-field-type-date a-nowrap')
    All_closing_dates = []
    for thing in closing_date:
        try:
            All_closing_dates.append(thing.string)
        except:
            All_closing_dates.append('NAN')
            continue
    closing_date_clean=[]
    for thing in All_closing_dates:
        y=thing.replace('Jan ','2018-01-')
        y2=y.replace('Feb ','2018-02-')
        y3=y2.replace('Mar ','2018-03-')
        y4=y3.replace('Apr ','2018-04-')
        y5=y4.replace('May ','2018-05-')
        y6=y5.replace('Jun ','2018-06-')
        y7=y6.replace('Jul ','2018-07-')
        y8=y7.replace('Aug ','2018-08-')
        y9=y8.replace('Sep ','2018-09-')
        y10=y9.replace('Oct ','2018-10-')
        y11=y10.replace('Nov ','2018-11-')
        y12=y11.replace('Dec ','2018-12-')
        closing_date_clean.append(y12)
    
    closing_date_date_time=[]
    for thing in closing_date_clean:
        try:
            y2=datetime.strptime(thing,'%Y-%m-%d')
            closing_date_date_time.append(y2)
        except:
            closing_date_date_time.append(thing)
    return closing_date_date_time

In [219]:
#getting the duration of the movie in theaters
def subtract_closing_from_opening_date (release_date_date_time,closing_date_date_time):
    i=0
    subtracted_o_c=[]
    for thing in range(len(closing_date_date_time)):
        try:
            subtracted=closing_date_date_time[i]-release_date_date_time[i]
            subtracted_o_c.append(subtracted)
            i+=1
        except:
            subtracted_o_c.append('-')
            i+=1
    days_in_theatre=[]
    for thing in subtracted_o_c:
        try:
            days_in_theatre.append(int(str(thing).split()[0]))
        except:
            days_in_theatre.append('-')
            
    #changes dashes to NANs
    days_in_theatre_with_none=[]
    for item in days_in_theatre:
        if type(item)==int:
            days_in_theatre_with_none.append(item)
        else:
            days_in_theatre_with_none.append('NAN')
            continue

In [220]:
#making the tuple to put into 
def make_tuple_func (All_names, gross_numbers_with_none, opening_numbers_with_none):
    i=0
    All_data_mojo=[]
    for thing in range(len(All_names)):
        new_tuple = (All_names[i],opening_numbers_with_none[i],gross_numbers_with_none[i],days_in_theatre_with_none)
        All_data_mojo.append(new_tuple)
        i+=1
    return All_data_mojo

In [221]:
#inserts tuple into sql table
def insert_func (cnx, cursor, All_data_mojo):
    insert_stm = """INSERT INTO mojo_data (title, days_in_theater, opening_revenue, gross_revenue, days_in_theatre_with_none) 
    VALUES (%s, %s, %s, %s)"""
    cursor.executemany(insert_stm, All_data_mojo)
    cnx.commit()

In [222]:
def process (url):
    soup = request_func(url)
    All_names = movie_titles(soup)
    gross_numbers_with_none = total_gross(soup)
    opening_numbers_with_none = opening_weekend_gross(soup)
    release_date_date_time = release_dates(soup)
    closing_date_date_time = closing_dates(soup)
    days_in_theatre_with_none = subtract_closing_from_opening_date(release_date_date_time,closing_date_date_time)
    All_data_mojo=make_tuple_func(All_names, gross_numbers_with_none, opening_numbers_with_none)
    insert_func (cnx, cursor, All_data_mojo)
    return All_data_mojo

    
    